In [9]:
import openai; openai.api_key = open('open_ai.key', 'r').read()
import os
os.environ['HTTP_PROXY'] = "http://proxy.mei.co.jp:8080"
os.environ['HTTPS_PROXY'] = "http://proxy.mei.co.jp:8080"

In [10]:
def chat_complete(messages, max_tokens=1024, temperature=0.5, debug=False):
    stop = False
    res = ''
    if debug:
        for m in messages:
            print(m["role"],'\t',"-"*30)
            print(m["content"])
            print()
    total_tokens = 0
    last_token = 0
    while not stop:
        success = False
        while not success:
            try:
                response = openai.ChatCompletion.create(
                    model="gpt-3.5-turbo-0301", messages=messages,
                    temperature=temperature, max_tokens=max_tokens
                )
                success = True
                res += response.choices[0].message.content
                total_tokens += response.usage.total_tokens
                last_token = response.usage.total_tokens
                # if debug:
                #     print(json.dumps(response.usage),response.choices[0].message.content.replace('\n',' '))
                if response.choices[0].finish_reason == 'stop':
                    stop = True
                else:
                    messages.append({"role": 'user', "content": response.choices[0].message.content})
            except Exception as e:
                print(e)
                # time.sleep(5)
    if debug:
        print('assistant','\t',"-"*30)
        print(res, '\n')
        print('usage','\t',"-"*30)
        print('last:', last_token, 'total:', total_tokens)
    return res

In [11]:
def translate(text,debug=False):
    messages = [
        {'role': 'system', 'content': "あなたの役割は論文を翻訳することです。"},
        {'role': 'user', 'content': text},
    ]
    res = chat_complete(messages,max_tokens=4000, temperature=0., debug=debug)
    return(res)

In [12]:
import tiktoken
def num_tokens_from_messages(messages, model="gpt-3.5-turbo-0301"):
    """Returns the number of tokens used by a list of messages."""
    try:
        encoding = tiktoken.encoding_for_model(model)
    except KeyError:
        encoding = tiktoken.get_encoding("cl100k_base")
    if model == "gpt-3.5-turbo-0301":  # note: future models may deviate from this
        num_tokens = 0
        for message in messages:
            num_tokens += 4  # every message follows <im_start>{role/name}\n{content}<im_end>\n
            for key, value in message.items():
                num_tokens += len(encoding.encode(value))
                if key == "name":  # if there's a name, the role is omitted
                    num_tokens += -1  # role is always required and always 1 token
        num_tokens += 2  # every reply is primed with <im_start>assistant
        return num_tokens
    else:
        raise NotImplementedError(f"""num_tokens_from_messages() is not presently implemented for model {model}.
    See https://github.com/openai/openai-python/blob/main/chatml.md for information on how messages are converted to tokens.""")


In [13]:
enc = tiktoken.encoding_for_model("gpt-3.5-turbo-0301")
def tik(text):
    return len(enc.encode(text))

In [17]:
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

text = "This is a sample text. It contains multiple sentences. Let's see if it works!"
max_tokens = 15

sentences = sent_tokenize(text)
chunks = []
chunk = ""
for sentence in sentences:
    print(sentence, tik(chunk), tik(sentence))
    if (tik(chunk)+tik(sentence)) <= max_tokens:
    # if len(chunk.split()) + len(sentence.split()) <= max_tokens:
        chunk += " " + sentence
    else:
        chunks.append(chunk.strip())
        chunk = sentence
if chunk:
    chunks.append(chunk.strip())

print(chunks)

This is a sample text. 0 6
It contains multiple sentences. 6 5
Let's see if it works! 11 7
['This is a sample text. It contains multiple sentences.', "Let's see if it works!"]
